In [61]:
import pandas as pd
from graphviz import Digraph
%matplotlib inline

In [49]:
users = [
    {"id": 0, "name": "Hero"},
    {"id": 1, "name": "Dunn"},
    {"id": 2, "name": "Sue"},
    {"id": 3, "name": "Chi"},
    {"id": 4, "name": "Thor"},
    {"id": 5, "name": "Clive"},
    {"id": 6, "name": "Hicks"},
    {"id": 7, "name": "Devin"},
    {"id": 8, "name": "Kate"},
    {"id": 9, "name": "Klein"}
]
assert len(users) == 10

In [51]:
friendships = [
    (0, 1),
    (0, 2),
    (1, 2),
    (1, 3),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 6),
    (5, 7),
    (6, 8),
    (7, 8),
    (8, 9)
]
assert len(friendships) >= len(users) -1

In [53]:
for user in users:
    user['friends'] = []

In [54]:
users

[{'friends': [], 'id': 0, 'name': 'Hero'},
 {'friends': [], 'id': 1, 'name': 'Dunn'},
 {'friends': [], 'id': 2, 'name': 'Sue'},
 {'friends': [], 'id': 3, 'name': 'Chi'},
 {'friends': [], 'id': 4, 'name': 'Thor'},
 {'friends': [], 'id': 5, 'name': 'Clive'},
 {'friends': [], 'id': 6, 'name': 'Hicks'},
 {'friends': [], 'id': 7, 'name': 'Devin'},
 {'friends': [], 'id': 8, 'name': 'Kate'},
 {'friends': [], 'id': 9, 'name': 'Klein'}]

In [63]:
!pwd

/Users/dave/repos/temp/data science book


In [55]:
for i, j in friendships:
    users[i]['friends'].append(j)
    users[j]['friends'].append(i)

In [62]:
pd.DataFrame.from_dict(users)

,friends,id,name
0,"[1, 2]",0,Hero
1,"[0, 2, 3]",1,Dunn
2,"[0, 1, 3]",2,Sue
3,"[1, 2, 4]",3,Chi
4,"[3, 5]",4,Thor
5,"[4, 6, 7]",5,Clive
6,"[5, 8]",6,Hicks
7,"[5, 8]",7,Devin
8,"[6, 7, 9]",8,Kate
9,[8],9,Klein


In [8]:
def number_of_friends(user):
    """how many friends does _user_ have?"""
    return len(user["friends"])

In [9]:
total_connections = sum(number_of_friends(user) for user in users)

In [10]:
total_connections

24